In [1]:
# Imports
import torch
from torch import nn
from src.utils.preprocess_utils import midi_to_note_indices, notes_to_midi
import numpy as np
import torch.nn.functional as F
from mido import MidiFile, MidiTrack, Message
from datetime import datetime

In [2]:
# Define model architecture
class LSTMModel(nn.Module):
    def __init__(self, vocab_size, embedding_size, hidden_size, num_layers):
        super(LSTMModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_size)
        self.lstm = nn.LSTM(embedding_size, hidden_size, num_layers, batch_first=True, dropout=0.2)
        self.fc = nn.Linear(hidden_size, vocab_size)
        
    def forward(self, x, hidden=None):
        # x: [batch_size, sequence_length]
        x = x.long()
        embedded = self.embedding(x)  # [batch_size, sequence_length, embedding_size]
        lstm_out, hidden = self.lstm(embedded, hidden)  # [batch_size, sequence_length, hidden_size]
        output = self.fc(lstm_out)  # [batch_size, sequence_length, vocab_size]
        
        return output

In [3]:
# Load model
model_name = "2025-01-04_22-07-28_e64_h128_l2_sl50"
model = torch.load(f'weights/model_2/{model_name}')
model.eval()

C:\Users\janikbalint\AppData\Local\Temp\ipykernel_12860\3629463313.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(f'weights/model_2/{model_name}')


LSTMModel(
  (embedding): Embedding(85, 64)
  (lstm): LSTM(64, 128, num_layers=2, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=128, out_features=85, bias=True)
)

In [4]:
# Load input
input = midi_to_note_indices("data/input.mid", 24, 108)
sequence_length = 200  # Desired length of the generated sequence
temperature = 0.3
print(input)

[24, 26, 31, 36]


In [5]:
def generate_sequence(model, start_sequence, sequence_length, num_notes, temperature=1.0):
    model.eval()

    start_tensor = torch.tensor(start_sequence, dtype=torch.long).unsqueeze(0)  # [1, sequence_length, input_size]

    generated_sequence = list(start_sequence)

    for i in range(sequence_length):
        with torch.no_grad():
            output = model(start_tensor)  # [1, i, output_size]
            logits = output.squeeze(0)  # Remove the batch dimension: [i, output_size]

            # Temperature scaling
            scaled_logits = logits
            if temperature != 0:
                scaled_logits = logits / temperature

            # Apply softmax to get probabilities for the next note
            output_probs = F.softmax(scaled_logits, dim=-1)

            # Sample the next note based on the probabilities
            next_note = torch.multinomial(output_probs[-1], 1).item()

            # Append the predicted note to the sequence and this sequence's one-hot
            # encoded format will be the input for the next note's generation
            generated_sequence.append(next_note)
            generated_sequence_tensor = np.array(generated_sequence)
            start_tensor = torch.tensor(generated_sequence_tensor, dtype=torch.long).unsqueeze(0)  # Add batch dimension
    
    return generated_sequence

generated_sequence = generate_sequence(model.cpu(), input, sequence_length, num_notes=85, temperature=temperature)

print("Generated Sequence:")
print(generated_sequence)

Generated Sequence:
[24, 26, 31, 36, 28, 8, 43, 40, 19, 7, 19, 38, 35, 42, 45, 41, 29, 33, 38, 34, 29, 32, 20, 8, 20, 32, 38, 29, 32, 53, 52, 56, 53, 35, 52, 50, 49, 46, 43, 40, 52, 47, 43, 20, 45, 41, 44, 43, 40, 45, 47, 42, 38, 43, 48, 45, 43, 42, 40, 43, 41, 40, 43, 39, 42, 40, 38, 36, 21, 33, 40, 36, 31, 35, 19, 31, 43, 34, 40, 38, 31, 43, 14, 8, 34, 31, 8, 20, 8, 56, 38, 41, 53, 47, 49, 50, 35, 41, 38, 56, 53, 51, 52, 50, 49, 34, 48, 46, 47, 45, 19, 43, 31, 38, 44, 42, 21, 9, 39, 40, 42, 45, 43, 41, 38, 40, 26, 38, 23, 36, 32, 35, 36, 38, 32, 35, 37, 40, 36, 36, 33, 28, 37, 34, 31, 28, 16, 40, 34, 43, 31, 37, 34, 58, 46, 51, 49, 53, 51, 50, 48, 56, 47, 44, 32, 51, 48, 49, 50, 41, 48, 45, 47, 49, 19, 43, 31, 47, 43, 38, 50, 46, 42, 38, 40, 43, 41, 39, 38, 40, 38, 36, 39, 35, 37, 43, 34, 31, 40, 36, 38, 42, 39, 36]


In [6]:
current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
file_path = f"generations/model_2/{current_datetime}_t{temperature:.1f}_w{sequence_length}_M_{model_name}.mid"

notes_to_midi(generated_sequence, file_path)

MIDI file saved to generations/model_2/2025-01-04_22-19-06_t0.3_w200_M_2025-01-04_22-07-28_e64_h128_l2_sl50.mid
